<a href="https://colab.research.google.com/github/pevu97/Jobb-Offer-Classifier/blob/main/notebook/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
!pip install beautifulsoup4

def get_job_details(job_id):
  job_id = job_id
  url = f"https://nofluffjobs.com/api/posting/{job_id}"

  headers = {
      "User-Agent": "Mozilla/5.0",
      "Accept": "application/json"
  }

  response = requests.get(url, headers=headers)

  return response.json()


def get_job_list(sentence):
  url = "https://nofluffjobs.com/api/search/posting"
  params = {
      "pageFrom": 1,
      "pageTo": 1,
      "pageSize": 20,
      "salaryCurrency": "PLN",
      "salaryPeriod": "month",
      "region": "pl",
      "language": "pl-PL",
      "withSalaryMatch": "true"
  }

  headers = {
      "Content-Type": "application/infiniteSearch+json",
      "User-Agent": "Mozilla/5.0"
  }

  payload = {
      "rawSearch": f"requirement={sentence}"
  }

  response = requests.post(url, headers=headers, params=params, json=payload)

  return response.json()


In [2]:
offer_list = get_job_list("python")
type(offer_list)

dict

In [3]:
offer_list['postings'][215]['id']

'lead-full-stack-software-engineer-python-pandas-square-one-resources-west-pomeranian'

In [4]:
jobs_titles = []
for offer in offer_list['postings']:
  jobs_titles.append(offer['id'])

In [5]:
job_details = get_job_details(jobs_titles[0])

In [6]:
keywords = [
    "python", "java", "javascript", "react", "django", "sql", "node", "devops", "qa",
    "junior", "mid", "entry", "trainee", "intern"
]


title_list = []
for key in keywords:
  offer_list = get_job_list(key)['postings']
  for offer in offer_list:
    title_list.append(offer['id'])
len(title_list)

2020

In [7]:
title_list[1]

'senior-python-full-stack-developer-link-group-lower-silesian'

In [8]:
job_details = []

for title in title_list:
  job_details.append(get_job_details(title))


In [9]:
len(job_details)

2020

In [10]:
from collections import Counter

labels = [offer['basics']['seniority'][0].lower() for offer in job_details if offer.get('basics', {}).get('seniority')]
print(Counter(labels))

Counter({'senior': 932, 'mid': 696, 'junior': 177, 'trainee': 144, 'expert': 71})


In [11]:
jobs = {'senior':[], 'mid':[], 'junior':[]}

In [12]:
for offer in job_details:
  if offer.get('basics', {}).get('seniority'):
    if 'senior' in offer['basics']['seniority'][0].lower():
      jobs['senior'].append(offer)
    if 'mid' in offer['basics']['seniority'][0].lower():
      jobs['mid'].append(offer)
    if 'junior' in offer['basics']['seniority'][0].lower():
      jobs['junior'].append(offer)
    if 'trainee' in offer['basics']['seniority'][0].lower():
      jobs['junior'].append(offer)
    if 'expert' in offer['basics']['seniority'][0].lower():
      jobs['senior'].append(offer)

In [13]:
len(jobs['senior'])

1003

In [62]:
type(jobs['senior'][0])

dict

In [14]:
jobs['senior'][0]['title']

'Senior Python Full-Stack Developer'

In [16]:

from bs4 import BeautifulSoup


text = jobs['senior'][0]['requirements']['description']

soup = BeautifulSoup(text, 'html.parser')
clean_text = soup.get_text()
print(clean_text)


5+ years of commercial experience in web-based application development
Strong Python skills, including Django, Flask or FastAPI
Very good JavaScript (ES6+), HTML5, CSS3
Experience with React or Angular
Understanding of OOP and functional programming
Experience with REST APIs, microservices, SQL databases
Knowledge of Clean Code, unit testing, CI/CD
Experience with Git, Docker, and cloud deployments (AWS or Azure)
Excellent English skills (spoken and written)
Experience in agile teams and international projects

Nice to have:

Experience with Kubernetes
Certifications in Python, SQL, AWS or Azure
German or another European language



In [17]:
jobs['senior'][0]['requirements']['musts'][0]

{'value': 'Python', 'type': 'main'}

In [18]:
for key in jobs['senior'][0]['requirements']['musts'][0]:
  print(jobs['senior'][0]['requirements']['musts'][0][key])

Python
main


In [51]:
new_offers = {'senior':['senior'], 'mid':['mid' ], 'junior':[ 'junior']}

In [59]:
for offer in jobs['senior']:
  if offer['requirements']['description']:
    soup = BeautifulSoup(offer['requirements']['description'], 'html.parser')
    clean_text = soup.get_text()


{'senior': 'Senior Python Full-Stack Developer',
 'mid': ['mid'],
 'junior': ['junior']}

In [71]:
from bs4 import BeautifulSoup

def clean_html(html_str):
    return BeautifulSoup(html_str or "", "html.parser").get_text(separator=" ")

sample = jobs["senior"][0]
text = f"{sample['title']}\n{clean_html(sample['requirements']['description'])}\n{clean_html(sample['details']['description'])}\n{clean_html(sample['requirements']['musts'][0])}"
print(text)


TypeError: unhashable type: 'slice'

In [72]:
from bs4 import BeautifulSoup
import pandas as pd

def clean_html(html_str):
    return BeautifulSoup(html_str or "", "html.parser").get_text(separator=" ")

dataset = []

for label, offers in jobs.items():
    for offer in offers:
        title = offer.get("title", "")

        desc_html = offer.get("details", {}).get("description", "")
        req_html = offer.get("requirements", {}).get("description", "")

        musts = offer.get("requirements", {}).get("musts", [])
        nices = offer.get("requirements", {}).get("nices", [])

        must_skills = " ".join([m.get("value", "") for m in musts])
        nice_skills = " ".join([n.get("value", "") for n in nices])

        desc_text = clean_html(desc_html)
        req_text = clean_html(req_html)

        full_text = f"{title}\n{desc_text}\n{req_text}\nMust have: {must_skills}\nNice to have: {nice_skills}"

        dataset.append({
            "text": full_text,
            "label": label
        })

# Konwersja do DataFrame i zapis
df = pd.DataFrame(dataset)
df.to_csv("job_offers_dataset.csv", index=False)
print("✅ Zapisano plik job_offers_dataset.csv")


✅ Zapisano plik job_offers_dataset.csv


In [79]:
df['label'].value_counts()

,count
label,
senior,1003
mid,696
junior,321


In [89]:
min_count = 321

In [92]:
senior = df[df['label'] == 'senior'].sample(n=min_count, random_state=42)
mid = df[df['label'] == 'mid'].sample(n=min_count, random_state=42)
junior = df[df['label'] == 'junior'].sample(n=min_count, random_state=42)

In [97]:
new_df = pd.concat([senior, mid, junior], ignore_index=True)


In [99]:
new_df['label'].value_counts()

,count
label,
senior,321
mid,321
junior,321
